In [2]:
library(Seurat)
library(harmony)
library(SeuratData)
library(ggplot2)
library(patchwork)
library(mclust)
library(aricode)

In [4]:
create_obj<-function(data_file,meta_file,coord_file){
    data=read.csv(data_file,header=T,row.names=1,sep=',')
    meta=read.csv(meta_file,header=T,row.names=1,sep=',')
    coord=read.csv(coord_file,header=T,row.names=1,sep=',')
    if(dim(data)[1]==dim(meta)[1]){data=t(data)}
    
    data=as(as.matrix(data),'dgCMatrix')
    obj<-CreateSeuratObject(counts=data,min.cells=0, min.features=0,meta.data=meta)
    VariableFeatures(obj)<-rownames(obj)
    obj@assays$RNA@scale.data=as.matrix(count)
    obj<-ScaleData(obj)
    obj<-RunPCA(obj,npcs=50)
    obj<-RunUMAP(obj,reduction='pca',dims=1:50)
    obj@images$image<-new(
        Class = 'SlideSeq',
        assay = "Spatial",
        key = "image_",
        coordinates = coord
    )
    return(obj)
}
clustering<-function(obj,embed_file){
    embed1=read.csv(embed_file,header=T,row.names=1)
    obj[['GTT']]<-CreateDimReducObject(embeddings=as.matrix(embed1), key = "GTT_", assay = DefaultAssay(obj))
    obj <-RunUMAP(obj, reduction = "GTT",dims=1:dim(embed1)[2],reduction.name="GTT.umap")
    obj<-FindNeighbors(obj, dims=1:dim(embed1)[2],reduction="GTT")
    obj<-FindClusters(obj, resolution =res1)
    obj$gtt=obj$seurat_clusters
    return(obj)
}